<a href="https://colab.research.google.com/github/crystaljwang/tm10007_group_3/blob/validation/ML_Geanne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn import metrics


In [60]:

# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git


# Run this to use from colab environment
!git clone https://github.com/jveenland/tm10007_ml.git

%cd /content/tm10007_ml/worcgist 

dir = Path('.') / 'GIST_radiomicFeatures.csv'
data = pd.read_csv(dir, index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

  Preparing metadata (setup.py) ... done
fatal: destination path 'tm10007_ml' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/tm10007_ml/worcgist'
/content/drive/MyDrive/Github/TM10007_ML/worcgist
The number of samples: 246
The number of columns: 494


In [61]:
##DATA INFORMATION
#get information to decide what scaler to use
#get information to decide what feature selection to use
#get information to decide what classifier to use

In [73]:
##PREPROCESSING

# Replace label values from string to binary
data['label'] = data['label'].replace({'GIST': 1, 'non-GIST': 0})

# Separate the features and labels
X = data.drop(['label'], axis=1)
y = data['label']


In [74]:
##TRAIN-TEST SET

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
##SCALING

# Data scaling
scaler = StandardScaler()  #define scaler
X_train = scaler.fit_transform(X_train)  #fit scaler on train set
X_test = scaler.transform(X_test)  #apply fitted scaler on test set

In [76]:
##FEATURE SELECTION

# Perform feature selection on the training set
selector = SelectKBest(f_classif, k=6)
X_train = selector.fit_transform(X_train, y_train)

# Apply the same feature selection to the test set
X_test = selector.transform(X_test)


/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 77  83  84  90  97 103 110 467 474 475 480 487 488] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [77]:
##k-fold validation
from sklearn.model_selection import KFold

# Initialize the k-fold and set the number of splits
kf = KFold(n_splits=8)

model = RandomForestClassifier()
fold = []
acc_list = []
auc_list = []
cm_list = []

# Split the data into training and testing sets
for train_index, test_index in kf.split(X_train):
    fold.append(test_index)
    X_train2, X_val = X_train[train_index], X_train[test_index] #features for training  and testing 
    y_train2, y_val = y_train[train_index], y_train[test_index] # targets for training and testing 
    model.fit(X_train2, y_train2)
    y_pred = model.predict(X_val)
    acc_list.append(metrics.accuracy_score(y_val, y_pred))
    fpr, tpr, _thresholds = metrics.roc_curve(y_val, y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr),2))
    cm_list.append(confusion_matrix(y_val, y_pred))


##VALIDATION
# Plot Model validation
result_df = pd.DataFrame({'Model':fold, 'Accuracy': acc_list, 'AUC': auc_list})
result_df

##final Validation




,Model,Accuracy,AUC
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0.400000,0.40
1,"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3...",0.480000,0.49
2,"[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 6...",0.560000,0.56
3,"[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 8...",0.760000,0.75
4,"[100, 101, 102, 103, 104, 105, 106, 107, 108, ...",0.708333,0.72
5,"[124, 125, 126, 127, 128, 129, 130, 131, 132, ...",0.625000,0.66
6,"[148, 149, 150, 151, 152, 153, 154, 155, 156, ...",0.541667,0.56
7,"[172, 173, 174, 175, 176, 177, 178, 179, 180, ...",0.500000,0.54


In [ ]:
##CLASSIFIERS

# Prepare pipeline
model_pipeline = []
model_pipeline.append(LogisticRegression(solver='liblinear'))
model_pipeline.append(SVC())
model_pipeline.append(LinearSVC())
model_pipeline.append(KNeighborsClassifier())
model_pipeline.append(DecisionTreeClassifier())
model_pipeline.append(RandomForestClassifier())
model_pipeline.append(GaussianNB())

model_list = ['LogisticRegression', 'SVC', 'Linear SVC', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes']
acc_list = []
auc_list = []
cm_list = []

# Execute pipeline
for model in model_pipeline:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  acc_list.append(metrics.accuracy_score(y_test, y_pred))
  fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
  auc_list.append(round(metrics.auc(fpr, tpr),2))
  cm_list.append(confusion_matrix(y_test, y_pred))



In [ ]:
##VALIDATION
# Plot Model validation
result_df = pd.DataFrame({'Model':model_list, 'Accuracy': acc_list, 'AUC': auc_list})
result_df

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/Github/TM10007_ML/worcgist

# # Data loading functions. Uncomment the one you want to use
# from load_data import load_data
# #from worclipo.load_data import load_data
# #from worcliver.load_data import load_data
# #from ecg.load_data import load_data


# #load data
# data = load_data()
# print(f'The number of samples: {len(data.index)}')
# print(f'The number of columns: {len(data.columns)}')
# data.info()
